<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#News-Data" data-toc-modified-id="News-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>News Data</a></span><ul class="toc-item"><li><span><a href="#Import-News-Data-(15-mnths)" data-toc-modified-id="Import-News-Data-(15-mnths)-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Import News Data (15 mnths)</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Preprocessing</a></span><ul class="toc-item"><li><span><a href="#Filter-into-ESG-News" data-toc-modified-id="Filter-into-ESG-News-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Filter into ESG News</a></span></li><li><span><a href="#Fix-the-timestamp" data-toc-modified-id="Fix-the-timestamp-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Fix the timestamp</a></span></li></ul></li><li><span><a href="#Convert-into-vector-representation-(Fin-Bert)" data-toc-modified-id="Convert-into-vector-representation-(Fin-Bert)-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Convert into vector representation (Fin-Bert)</a></span><ul class="toc-item"><li><span><a href="#Use-ProsusAI/finbert" data-toc-modified-id="Use-ProsusAI/finbert-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Use ProsusAI/finbert</a></span></li><li><span><a href="#Use-yiyanghkust/finbert-pretrain" data-toc-modified-id="Use-yiyanghkust/finbert-pretrain-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Use yiyanghkust/finbert-pretrain</a></span></li></ul></li></ul></li><li><span><a href="#ESG-Data" data-toc-modified-id="ESG-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>ESG Data</a></span><ul class="toc-item"><li><span><a href="#Match-News-with-ESG-Controversy-Score" data-toc-modified-id="Match-News-with-ESG-Controversy-Score-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Match News with ESG Controversy Score</a></span></li><li><span><a href="#Training-and-Test-Set" data-toc-modified-id="Training-and-Test-Set-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Training and Test Set</a></span></li><li><span><a href="#Data-Preparation's-Done" data-toc-modified-id="Data-Preparation's-Done-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Data Preparation's Done</a></span></li></ul></li></ul></div>

In [1]:
import torch
from transformers import BertTokenizer, BertModel

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import svm
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB
from sklearn.model_selection import GridSearchCV
from datetime import datetime, timedelta

import pandas as pd
import string
import os
import warnings

pd.set_option('display.max_columns', 100)
warnings.filterwarnings('ignore')

# download FTSE100
fortune100_df = pd.read_csv('../data/DAX40.csv')
fortune100_df = fortune100_df.iloc[:100,:]
# fortune100_df.iloc[99, 1] = 'Cigna Group'

# News Data

## Import News Data (15 mnths)

In [5]:
news_df = pd.read_csv('../data/output_news_headline_DAX40.csv')
news_df.head()

,Unnamed: 0,versionCreated,text,storyId,sourceCode,firm
0,2022-03-09 21:02:00.662,2022-03-09 21:25:51.262000+00:00,Refinitiv Newscasts - U.S. Day Ahead: Inflatio...,urn:newsml:newswire.refinitiv.com:20220309:nRT...,NS:RTRS,ADSGn.DE
1,2022-03-09 09:43:35.000,2022-03-09 17:16:12+00:00,"UPDATE 3-Banks, automakers fuel strong comebac...",urn:newsml:newswire.refinitiv.com:20220309:nL3...,NS:RTRS,ADSGn.DE
2,2022-03-09 11:19:19.111,2022-03-09 17:01:39.061000+00:00,Refinitiv Newscasts - Adidas sees Russia hit i...,urn:newsml:newswire.refinitiv.com:20220309:nRT...,NS:RTRS,ADSGn.DE
3,2022-03-09 12:32:41.482,2022-03-09 17:01:06.673000+00:00,Refinitiv Newscasts - U.S. Morning Call: Globa...,urn:newsml:newswire.refinitiv.com:20220309:nRT...,NS:RTRS,ADSGn.DE
4,2022-03-09 16:03:13.000,2022-03-09 16:03:13+00:00,BUZZ-Nike climbs as rival Adidas' earnings for...,urn:newsml:newswire.refinitiv.com:20220309:nL3...,NS:RTRS,ADSGn.DE


In [6]:
news_df[['firm','sourceCode']].groupby('firm').count().sort_values('sourceCode', ascending=False).T

firm,DBKGn.DE,VOWG_p.DE,AIR.PA,RWEG.DE,MBGn.DE,BASFn.DE,SIEGn.DE,BMWG.DE,BAYGn.DE,ENR1n.DE,PSHG_p.DE,ADSGn.DE,DTEGn.DE,EONGn.DE,ALVG.DE,SAPG.DE,CONG.DE,IFXGn.De,DPWGn.DE,1COV.DE,FMEG.DE,MRCG.DE,HNKG.DE,VNAn.DE,PUMG.DE,FREG.DE,HEIG.DE,HFGG.DE,SY1G.DE,ZALG.DE,BEIG.DE,BNRGn.DE,MUVGn.DE,QIA.DE,SATG.DE,SHLG.DE,MTXGn.DE,DWNG.DE,LIN
sourceCode,3548,2509,2415,1389,1268,1181,1158,1116,1026,1012,904,772,714,601,596,480,467,457,438,416,364,342,337,335,332,318,307,302,298,286,284,272,260,252,243,221,201,85,57


## Preprocessing

### Filter into ESG News

In [7]:
# dummy dataframe of headline for sentiment result
esg_glossaries = pd.read_csv('../data/ESG Score Compile - Glossary.csv')                 

# remove unused row number
esg_glossaries = esg_glossaries.drop(['No'], axis=1)

# lower the glossaries
esg_glossaries['Keywords'] = esg_glossaries['Keywords'].str.lower()

# remove duplicates
esg_glossaries = esg_glossaries.drop_duplicates(['Keywords'])

In [8]:
def ESG_news_filter(company_df, text_col='text', inplace=True):
    datapath=os.getcwd()[:-len('notebook')] + "data/"
    ESG_data=pd.read_csv(datapath+'Categorized_ESG_data.csv')
    
    X=ESG_data['T'].str.lower()
    y=ESG_data['S']
    
    cv = CountVectorizer()
    features = cv.fit_transform(X)
    
    pre_trained_model = GaussianNB()
    pre_trained_model.fit(features.toarray(),y)
    
    X_company=company_df[text_col]
    Xf_company=cv.transform(X_company)
    
    ESG=pre_trained_model.predict(Xf_company.toarray())
    
    company_df['ESG']=ESG
    
    if inplace==True:
        return company_df[company_df['ESG']==1].drop(columns='ESG')
    else:
        return company_df


In [9]:
def filter_esg_news(df, esg_glossaries, column, verbose = False):

    ESG_news_filter = pd.DataFrame()
    cnt = 0
    for key in esg_glossaries.Keywords:
        cnt+=1
        ESG_news_filter = pd.concat([ESG_news_filter, df[df[column].str.contains(key)]])

        if verbose:
            if cnt % 10 == 0:
                print(cnt)
            
    return ESG_news_filter

In [10]:
def filter_companies(df):
    copy_df = df.copy()
    
    firm_names = [f.split(' ')[0].lower() for f in fortune100_df['Company']]
    copy_df['flag'] = 0
    
    for firm in firm_names:
        
        index = copy_df[copy_df['text_pp'].str.contains(firm)].index.tolist()
        copy_df.loc[index,'flag'] = 1
        
    return copy_df

In [11]:
def text_preprocessing_v2(df, types = 1):
    df_copy = df.copy()
    print('total news:',df_copy.shape[0])

    # assign a new seperate column text_pp for preprocessing
    df_copy['text_pp'] = df_copy.text

    # make the string lower
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.lower())

    # remove the punctuation
    # translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
    # df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.translate(translator))

    # remove unnecessary word
    # df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: x.replace('rpt',''))

    # delete unnecessary double-space
    df_copy['text_pp'] = df_copy['text_pp'].apply(lambda x: ' '.join(x.split()))
    
    if types == 1:
        
        # filter out only ESG news (model)
        df_copy = ESG_news_filter(df_copy, text_col='text_pp', inplace=True)
        print('filter out non-esg news:', df_copy.shape[0])
        
    else:
        
        # filter out only ESG news (conventional)
        df_copy = filter_esg_news(df_copy, esg_glossaries, 'text_pp')
        print('filter out non-esg news:', df_copy.shape[0])
        
    # remove duplicates of headers
    df_copy = df_copy.drop_duplicates(['text_pp'])
    print('preprocessing and drop duplicates news:', df_copy.shape[0])
    
    df_copy = df_copy.drop(['Unnamed: 0'], axis=1)
    
    df_copy = filter_companies(df_copy)
    print('contain firm name #rows:', df_copy[df_copy['flag'] == 1].shape[0])

    return df_copy[df_copy['flag'] == 1].reset_index(drop=True)

In [12]:
ESG_news_df = text_preprocessing_v2(news_df, types = 1)
ESG_news_df2 = text_preprocessing_v2(news_df, types = 2)

total news: 27563
filter out non-esg news: 13988
preprocessing and drop duplicates news: 11601
contain firm name #rows: 11595
total news: 27563
filter out non-esg news: 4592
preprocessing and drop duplicates news: 3531
contain firm name #rows: 3528


In [13]:
# ESG_news_df = ESG_news_df[ESG_news_df['ESG'] == 1]
ESG_news_df = ESG_news_df.reset_index(drop=True)

In [14]:
# ESG_news_df = ESG_news_df[ESG_news_df['ESG'] == 1]
ESG_news_df2 = ESG_news_df2.reset_index(drop=True)

In [15]:
ESG_news_df_ML = ESG_news_df.copy()

In [16]:
ESG_news_df = ESG_news_df2.copy()

### Fix the timestamp

In [17]:
def timestamp_preprocessing(df, cols = 'versionCreated'):
    
    df_copy = df.copy()
    
    df_copy[cols] = df_copy[cols].apply(lambda x: x[:19])
    df_copy[cols] = pd.to_datetime(df_copy[cols])
    
    df_copy['year_month'] = df_copy[cols].apply(lambda x: datetime.strftime(x, '%Y-%m'))
    df_copy['year'] = df_copy[cols].apply(lambda x: datetime.strftime(x, '%Y'))

    return df_copy

In [18]:
ESG_news_df = timestamp_preprocessing(ESG_news_df)

In [19]:
ESG_news_df = ESG_news_df.sort_values(['firm','versionCreated'])
ESG_news_df.reset_index(drop=True, inplace = True)

## Convert into vector representation (Fin-Bert)

### Use ProsusAI/finbert

link: https://github.com/ProsusAI/finBERT/tree/master

In [20]:
# Pretrained language model
model_name = 'ProsusAI/finbert'
tokenizer = BertTokenizer.from_pretrained(model_name)
language_model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at ProsusAI/finbert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [21]:
print(ESG_news_df.shape[0])

3528


In [22]:
ESG_news_df.head(2)

,versionCreated,text,storyId,sourceCode,firm,text_pp,flag,year_month,year
0,2022-04-22 08:10:57,COVESTRO AG - EXTENDS FILM PRODUCTION IN GERMANY,urn:newsml:newswire.refinitiv.com:20220422:nFW...,NS:RTRS,1COV.DE,covestro ag - extends film production in germany,1,2022-04,2022
1,2022-04-22 08:19:43,BRIEF-Covestro Extends Film Production In Germany,urn:newsml:newswire.refinitiv.com:20220422:nFW...,NS:RTRS,1COV.DE,brief-covestro extends film production in germany,1,2022-04,2022


In [23]:
semantics_ftse100_df = pd.DataFrame()

for firm in ESG_news_df.firm.unique():
    
    for year in ESG_news_df[ESG_news_df.firm == firm].year.unique():
        
        copy_df = ESG_news_df[(ESG_news_df['firm'] == firm) & (ESG_news_df['year'] == year)].copy()
        esg_news = [news for news in copy_df['text_pp']]
        
        overall_semantics = []
        text_embeddings = []
        
        print(firm, year, 'started')

        for news in esg_news:
            # Sentiment extraction
            inputs = tokenizer.encode_plus(news, add_special_tokens=True, return_tensors='pt')
            outputs = language_model(**inputs)

            # Extract sentiment score from model output
            sentiment_score = outputs.pooler_output  

            # Text embedding
            text_embedding = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings

            overall_semantics.append(sentiment_score)
            text_embeddings.append(text_embedding)

        # Pooling operations
        semantics_avg_vector = torch.cat(overall_semantics, dim=0).mean(dim=0)  
        text_embedding_avg_vector = torch.cat(text_embeddings, dim=0).mean(dim=0) 
        
        
        temp_semantic_df = pd.DataFrame(semantics_avg_vector.detach()).T
        temp_semantic_df['year'] = year
        temp_semantic_df['firm'] = firm
        
        semantics_ftse100_df = pd.concat([semantics_ftse100_df, temp_semantic_df])

1COV.DE 2022 started
1COV.DE 2023 started
ADSGn.DE 2022 started
ADSGn.DE 2023 started
AIR.PA 2022 started
AIR.PA 2023 started
ALVG.DE 2022 started
ALVG.DE 2023 started
BASFn.DE 2022 started
BASFn.DE 2023 started
BAYGn.DE 2022 started
BAYGn.DE 2023 started
BEIG.DE 2022 started
BEIG.DE 2023 started
BMWG.DE 2022 started
BMWG.DE 2023 started
BNRGn.DE 2022 started
BNRGn.DE 2023 started
CONG.DE 2022 started
CONG.DE 2023 started
DBKGn.DE 2022 started
DBKGn.DE 2023 started
DPWGn.DE 2022 started
DPWGn.DE 2023 started
DTEGn.DE 2022 started
DTEGn.DE 2023 started
DWNG.DE 2022 started
ENR1n.DE 2022 started
ENR1n.DE 2023 started
EONGn.DE 2022 started
EONGn.DE 2023 started
FMEG.DE 2022 started
FMEG.DE 2023 started
FREG.DE 2022 started
FREG.DE 2023 started
HEIG.DE 2022 started
HEIG.DE 2023 started
HFGG.DE 2022 started
HFGG.DE 2023 started
HNKG.DE 2022 started
HNKG.DE 2023 started
IFXGn.De 2022 started
IFXGn.De 2023 started
LIN 2023 started
MBGn.DE 2022 started
MBGn.DE 2023 started
MRCG.DE 2022 started

In [24]:
semantics_ftse100_df = pd.concat([semantics_ftse100_df[['firm','year']], \
                                  semantics_ftse100_df.drop(['firm','year'], axis=1)], axis=1)

In [25]:
semantics_ftse100_df.head()

,firm,year,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,...,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,1COV.DE,2022,0.130479,0.505199,0.600378,-0.477656,-0.174471,0.499294,-0.600625,-0.636520,0.701751,0.540555,0.268798,-0.253827,0.107586,-0.665974,-0.199498,0.608968,0.484472,0.546881,-0.455419,0.419032,-0.334159,-0.415122,0.410068,-0.541447,-0.625590,-0.478560,0.384087,0.253635,0.066590,0.259534,0.352463,-0.620511,-0.264266,0.691462,0.460079,0.475398,-0.475996,0.240907,0.694434,0.533751,-0.339226,-0.616026,-0.414357,-0.358934,0.053088,0.686750,0.528950,-0.664712,...,0.355171,0.430726,0.751445,-0.223564,-0.745369,0.458091,0.760691,-0.220254,-0.432880,0.011229,-0.588437,0.380800,0.491015,0.021233,-0.523250,-0.493968,0.534216,0.531353,-0.315117,-0.424845,-0.411290,0.729978,0.693388,-0.173789,0.244618,-0.594603,0.156404,0.510495,-0.580846,0.240752,-0.636583,0.470717,-0.393042,0.403760,0.683070,0.651256,-0.002697,-0.372086,0.667601,0.316503,-0.196795,0.014009,-0.405986,0.297155,-0.619607,0.714014,-0.243243,0.335700,0.734606,0.024849
0,1COV.DE,2023,0.311119,0.681544,0.809474,-0.592760,-0.325240,0.669663,-0.746699,-0.805993,0.945332,0.516298,0.578142,-0.498506,-0.178987,-0.914193,-0.424431,0.842918,0.704549,0.701836,-0.627622,0.505984,-0.483349,-0.500072,0.547564,-0.765798,-0.853948,-0.612452,0.630197,0.180416,-0.125782,0.217388,0.467000,-0.834453,0.026611,0.895480,0.623583,0.525999,-0.618626,0.326149,0.915618,0.735308,-0.221153,-0.837093,-0.501060,-0.790584,-0.121856,0.901812,0.545600,-0.879711,...,0.675327,0.633764,0.990463,-0.079994,-0.893889,0.664763,0.980888,-0.369911,-0.648459,-0.104165,-0.819311,0.542255,0.685180,0.122597,-0.570915,-0.592896,0.764746,0.738711,-0.442167,-0.577002,-0.500019,0.952653,0.964394,-0.220209,0.330879,-0.826177,0.399355,0.500112,-0.776813,0.266326,-0.911662,0.723604,-0.541033,0.727139,0.717024,0.860038,0.009370,-0.580651,0.886062,0.643744,-0.148689,0.144850,-0.559157,0.179851,-0.706833,0.898148,-0.303378,0.469454,0.890642,-0.086419
0,ADSGn.DE,2022,-0.346749,-0.002502,-0.141569,0.135706,0.353608,0.097024,-0.032066,-0.205860,0.105043,-0.237411,-0.217469,0.426901,0.356096,-0.258442,0.258586,0.074039,0.088957,0.180499,-0.118204,0.249712,0.274465,0.371781,0.211265,-0.254681,-0.175135,0.370811,-0.237425,0.411039,0.383432,0.452075,-0.097054,-0.176692,-0.513129,0.364122,-0.334151,-0.255884,0.056133,-0.106999,0.378924,0.171222,-0.493156,-0.186640,0.293610,0.180465,0.374449,0.419876,-0.326793,-0.292623,...,-0.199953,0.066743,0.335701,-0.417782,-0.096500,0.024978,0.432765,0.420547,-0.096433,0.245783,-0.209278,-0.040470,0.121749,-0.283628,-0.075871,-0.412947,-0.007121,0.021476,0.133151,0.059132,0.352613,0.219945,0.082368,-0.324195,-0.189520,-0.252642,-0.293246,-0.332109,-0.144255,-0.307813,-0.080864,-0.170644,0.306919,-0.167017,-0.094815,0.231349,0.235620,0.264082,0.342874,-0.279262,-0.300725,-0.193226,0.398977,0.402939,-0.592786,0.497523,-0.205579,-0.367670,0.285578,0.283730
0,ADSGn.DE,2023,-0.279547,-0.344155,-0.515682,0.388854,0.359438,-0.240983,-0.184838,0.190240,-0.239913,-0.274135,-0.530859,0.756352,0.166015,0.013370,0.140225,-0.078657,-0.143189,-0.131080,0.204559,0.435426,0.265705,0.830945,-0.126563,0.131469,0.205743,0.783439,-0.465674,0.110721,0.163432,0.251913,-0.121745,0.239390,-0.283639,-0.059717,-0.699071,-0.317708,0.373950,-0.054267,-0.073685,-0.159548,-0.160112,0.192200,0.259652,0.397576,0.412245,-0.048476,-0.775661,0.158626,...,-0.416944,-0.242811,-0.001287,-0.182686,-0.060078,-0.331219,0.067933,0.718390,0.236790,0.270953,0.017236,-0.345481,-0.161474,-0.121413,-0.145725,-0.132234,-0.373290,-0.284510,0.182462,0.354712,0.827443,-0.117606,-0.241041,-0.169174,-0.424296,0.133933,-0.494146,-0.826807,0.192852,-0.636804,0.234599,-0.500917,0.602881,-0.457889,-0.0592

In [26]:
semantics_ftse100_df = semantics_ftse100_df.reset_index(drop=True)

In [27]:
semantics_ftse100_df['firm'] = semantics_ftse100_df['firm'].astype(str)
semantics_ftse100_df['year'] = semantics_ftse100_df['year'].astype(int)

In [28]:
semantics_ftse100_df.to_csv('../data/semantics_DAX40_esg_filter_contains.csv', index=None)

### Use yiyanghkust/finbert-pretrain

link: https://github.com/yya518/FinBERT/blob/master/finetune.ipynb

In [66]:
# Pretrained language model
model_name = 'yiyanghkust/finbert-pretrain'
tokenizer = BertTokenizer.from_pretrained(model_name)
language_model = BertModel.from_pretrained(model_name)

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [67]:
semantics_ftse100_v2_df = pd.DataFrame()

for firm in ESG_news_df.firm.unique():
    
    for year in ESG_news_df[ESG_news_df.firm == firm].year.unique():
        
        copy_df = ESG_news_df[(ESG_news_df['firm'] == firm) & (ESG_news_df['year'] == year)].copy()
        esg_news = [news for news in copy_df['text_pp']]
        
        overall_semantics = []
        text_embeddings = []
        
        print(firm, year, 'started')

        for news in esg_news:
            # Sentiment extraction
            inputs = tokenizer.encode_plus(news, add_special_tokens=True, return_tensors='pt')
            outputs = language_model(**inputs)

            # Extract sentiment score from model output
            sentiment_score = outputs.pooler_output  

            # Text embedding
            text_embedding = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings

            overall_semantics.append(sentiment_score)
            text_embeddings.append(text_embedding)

        # Pooling operations
        semantics_avg_vector = torch.cat(overall_semantics, dim=0).mean(dim=0)  
        text_embedding_avg_vector = torch.cat(text_embeddings, dim=0).mean(dim=0) 
        
        
        temp_semantic_df = pd.DataFrame(semantics_avg_vector.detach()).T
        temp_semantic_df['year'] = year
        temp_semantic_df['firm'] = firm
        
        
        semantics_ftse100_v2_df = pd.concat([semantics_ftse100_v2_df, temp_semantic_df])

AAF.L 2022 started
ABDN.L 2022 started
ABDN.L 2023 started
ABF.L 2022 started
ABF.L 2023 started
AHT.L 2022 started
ANTO.L 2022 started
ANTO.L 2023 started
AUTOA.L 2022 started
AV.L 2022 started
AV.L 2023 started
AZN.L 2022 started
AZN.L 2023 started
BAES.L 2022 started
BAES.L 2023 started
BARC.L 2022 started
BARC.L 2023 started
BATS.L 2022 started
BATS.L 2023 started
BDEV.L 2022 started
BDEV.L 2023 started
BEZG.L 2022 started
BEZG.L 2023 started
BKGH.L 2022 started
BKGH.L 2023 started
BLND.L 2022 started
BMEB.L 2022 started
BNZL.L 2022 started
BNZL.L 2023 started
BP.L 2022 started
BP.L 2023 started
BRBY.L 2022 started
BRBY.L 2023 started
BT.L 2022 started
BT.L 2023 started
CCH.L 2022 started
CPG.L 2022 started
CRDA.L 2022 started
CRDA.L 2023 started
CRH.L 2022 started
CRH.L 2023 started
CTEC.L 2022 started
DCC.L 2022 started
DCC.L 2023 started
DGE.L 2022 started
DGE.L 2023 started
ENT.L 2022 started
ENT.L 2023 started
EXPN.L 2022 started
EXPN.L 2023 started
FLTRF.L 2022 started
FLTRF.

In [73]:
semantics_ftse100_v2_df = pd.concat([semantics_ftse100_v2_df[['firm','year']], semantics_ftse100_v2_df.drop(['firm','year'], axis=1)], axis=1)

In [110]:
semantics_ftse100_v2_df = semantics_ftse100_v2_df.reset_index(drop=True)

In [111]:
semantics_ftse100_v2_df.head()

,firm,year,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,...,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767
0,AAF.L,2022,0.997741,0.029028,-0.997513,0.334838,0.712012,0.019522,-0.644898,-0.072796,0.963206,-0.093572,-0.001303,0.731549,-0.999912,-0.004749,0.135107,-0.198207,-0.201953,-0.032121,-0.130995,-0.713439,0.684404,0.995149,0.999928,0.845180,0.402251,0.999958,0.869724,0.113720,0.024483,0.048140,0.015801,0.037154,-0.117185,-0.368511,-0.170509,0.999451,-0.266585,-0.251341,0.215471,-0.085773,0.122216,0.095172,0.242770,0.109276,-0.183281,-0.146718,0.091036,0.007633,...,0.237865,-0.278159,0.129780,-0.354730,-0.868910,-0.989849,-0.918077,0.158916,-0.999907,0.095612,-0.060056,-0.009567,-0.999251,-0.215684,0.098297,0.003679,0.136678,0.166769,0.103936,0.123942,0.125368,-0.213877,-0.018230,0.342590,0.999801,-0.978898,0.234214,0.064304,-0.051108,-0.016347,-0.976152,-0.998102,-0.073052,0.843195,0.079209,-0.249616,0.998655,-0.999695,0.837114,0.050972,0.230975,-0.072215,-0.746568,0.092619,0.279348,-0.166760,-0.989930,0.589905,0.584731,0.049186
1,ABDN.L,2022,0.996819,0.095085,-0.998613,0.568138,0.788253,0.214046,-0.870445,-0.102602,0.995655,-0.104370,-0.010163,0.719127,-0.999712,-0.030351,0.018541,-0.023877,-0.125272,0.017154,-0.203656,-0.914491,0.387899,0.997920,0.999971,0.984138,0.028084,0.999976,0.989630,0.236592,0.203242,-0.079100,0.059357,-0.053562,-0.015643,-0.200015,-0.061069,0.999921,-0.195509,-0.217696,0.106379,-0.054508,0.093138,0.118872,0.773654,-0.009733,-0.204403,-0.086836,-0.064186,-0.057688,...,0.146257,-0.199227,0.271625,-0.205517,-0.871871,-0.997547,-0.989301,-0.066180,-0.999936,0.074904,0.086331,-0.075209,-0.999763,-0.203907,-0.062041,0.204395,0.104943,0.163442,0.291025,0.097555,-0.130973,-0.060302,-0.204357,0.222007,0.999965,-0.993112,0.013240,0.028441,-0.043148,-0.123588,-0.983625,-0.998408,-0.512533,0.776308,0.048624,0.064667,0.999083,-0.999676,0.922865,0.007692,-0.067436,0.098027,-0.872003,0.068767,0.307862,-0.002922,-0.974779,0.810444,0.785232,0.035567
2,ABDN.L,2023,0.989547,0.150536,-0.999749,0.047239,0.954630,0.151744,-0.988769,-0.238919,0.999199,-0.109350,0.035160,0.317401,-0.999984,0.089074,0.122602,-0.010561,-0.106659,-0.108626,-0.137339,-0.985335,0.619444,0.991225,0.999954,0.963679,0.483892,0.999970,0.997173,0.081734,0.150592,0.106016,0.019440,-0.013757,0.105857,-0.004731,-0.060471,0.999673,-0.262355,-0.090079,0.232048,0.031951,0.114128,0.285968,0.906139,0.158564,-0.098235,-0.103863,0.127101,-0.054738,...,0.133307,-0.236596,0.381947,-0.681377,-0.712286,-0.999416,-0.994456,-0.067077,-0.999947,0.041724,0.100697,-0.067905,-0.999445,-0.163924,-0.038487,0.162933,0.374952,0.102534,0.395389,0.173159,-0.021496,-0.063033,-0.181556,0.270891,0.999869,-0.969155,-0.031581,0.089514,-0.100322,-0.033030,-0.996256,-0.999651,0.011610,0.885921,0.143838,-0.125996,0.998634,-0.999975,0.993254,-0.014059,0.188017,-0.028435,-0.922217,0.145202,0.372554,0.211096,-0.996154,0.982039,0.958495,0.119671
3,ABF.L,2022,0.995933,0.117803,-0.998486,0.447475,0.712604,0.120528,-0.818661,-0.153640,0.982410,-0.072859,0.171956,0.755239,-0.999852,-0.248553,0.235087,-0.139464,0.014628,0.037759,-0.195925,-0.886161,0.440845,0.996099,0.999915,0.961138,-0.022704,0.999972,0.984891,-0.000657,0.002406,-0.083749,0.252296,0.094640,0.018567,-0.224951,0.086159,0.999873,-0.084604,-0.142325,0.020323,0.074382,-0.117331,0.291986,0.767179,0.039035,-0.024188,-0.028403,0.112348,0.120512,...,0.279936,-0.172295,-0.015387,-0.437296,-0.847734,-0.991688,-0.982044,-0.090973,-0.999944,0.195029,0.160253,-0.076432,-0.999666,-0.080111,0.153318,0.100536,0.151402,0.064145,0.116562,0.234809,0.036705,-0.278851,-0.068793,0.410675,0.999925,-0.978843,-0.100703,0.045183,-0.187457,0.064230,-0.995666,-0.995277,-0.297842,0.683482,-0.007643,-0.026577,0.9

# ESG Data

In [29]:
def grade_calculation_og(score):
    val = 'A+'

    if score <= 0.083333:
        val = 'D-'
    elif 0.083333 < score <= 0.166666: 
        val = 'D'
    elif 0.166666 < score <= 0.250000: 
        val = 'D+'
    elif 0.250000 < score <= 0.333333: 
        val = 'C-'
    elif 0.333333 < score <= 0.416666: 
        val = 'C'
    elif 0.416666 < score <= 0.500000: 
        val = 'C+'
    elif 0.500000 < score <= 0.583333: 
        val = 'B-'
    elif 0.583333 < score <= 0.666666: 
        val = 'B'
    elif 0.666666 < score <= 0.750000: 
        val = 'B+'
    elif 0.750000 < score <= 0.833333: 
        val = 'A-'
    elif 0.833333 < score <= 0.916666: 
        val = 'A'

    return val

In [30]:
def grade_calculation(score):
    score = score / 100
    
    if score <= 0.250000:
        val = 'D'
    elif score <= 0.500000: 
        val = 'C'
    elif score <= 0.750000: 
        val = 'B'
    elif score > 0.75: 
        val = 'A'

    return val

In [31]:
ESG_df = pd.read_csv('../data/output_ESGControversyScore_DAX.csv')

In [32]:
ESG_df = ESG_df.iloc[:, 1:]
ESG_df['Date'] = ESG_df['Date'].astype(str)
ESG_df = ESG_df[ESG_df.Date != 'nan'].reset_index(drop=True)
ESG_df = timestamp_preprocessing(ESG_df, cols = 'Date')

ESG_df['Instrument'] = ESG_df['Instrument'].astype(str)
ESG_df['year'] = ESG_df['year'].astype(int)

In [33]:
ESG_df['ESG Controversies Score Simple'] = ESG_df['ESG Controversies Score'].apply(lambda x: grade_calculation(x))

In [35]:
ESG_df['ESG Controversies Score Simple'].unique()

array(['D', 'B', 'A', 'C'], dtype=object)

## Match News with ESG Controversy Score

In [36]:
semantics_ftse100_df['year'] = semantics_ftse100_df['year'].astype(int)

In [37]:
for idx in range(semantics_ftse100_df.shape[0]):
    
    f = semantics_ftse100_df.iloc[idx, 0]
    y = int(semantics_ftse100_df.iloc[idx, 1])
    
    print(f,y)
    
    copy_df = ESG_df[(ESG_df['year'] == y) & (ESG_df['Instrument'] == f)].copy().reset_index(drop=True)
    
    if copy_df.shape[0] != 0:
    
        semantics_ftse100_df.loc[idx, 'ESG Controversies Score'] = copy_df['ESG Controversies Score'][0]
        semantics_ftse100_df.loc[idx, 'ESG Controversies Score Grade'] = copy_df['ESG Controversies Score Grade'][0]
        
    else:
        
        semantics_ftse100_df.loc[idx, 'ESG Controversies Score'] = -1
        semantics_ftse100_df.loc[idx, 'ESG Controversies Score Grade'] = 'Unknown'

1COV.DE 2022
1COV.DE 2023
ADSGn.DE 2022
ADSGn.DE 2023
AIR.PA 2022
AIR.PA 2023
ALVG.DE 2022
ALVG.DE 2023
BASFn.DE 2022
BASFn.DE 2023
BAYGn.DE 2022
BAYGn.DE 2023
BEIG.DE 2022
BEIG.DE 2023
BMWG.DE 2022
BMWG.DE 2023
BNRGn.DE 2022
BNRGn.DE 2023
CONG.DE 2022
CONG.DE 2023
DBKGn.DE 2022
DBKGn.DE 2023
DPWGn.DE 2022
DPWGn.DE 2023
DTEGn.DE 2022
DTEGn.DE 2023
DWNG.DE 2022
ENR1n.DE 2022
ENR1n.DE 2023
EONGn.DE 2022
EONGn.DE 2023
FMEG.DE 2022
FMEG.DE 2023
FREG.DE 2022
FREG.DE 2023
HEIG.DE 2022
HEIG.DE 2023
HFGG.DE 2022
HFGG.DE 2023
HNKG.DE 2022
HNKG.DE 2023
IFXGn.De 2022
IFXGn.De 2023
LIN 2023
MBGn.DE 2022
MBGn.DE 2023
MRCG.DE 2022
MRCG.DE 2023
MTXGn.DE 2022
MUVGn.DE 2022
MUVGn.DE 2023
PSHG_p.DE 2022
PSHG_p.DE 2023
PUMG.DE 2022
QIA.DE 2022
QIA.DE 2023
RWEG.DE 2022
RWEG.DE 2023
SAPG.DE 2022
SAPG.DE 2023
SATG.DE 2022
SATG.DE 2023
SHLG.DE 2022
SHLG.DE 2023
SIEGn.DE 2022
SIEGn.DE 2023
SY1G.DE 2022
SY1G.DE 2023
VNAn.DE 2022
VNAn.DE 2023
VOWG_p.DE 2022
VOWG_p.DE 2023
ZALG.DE 2022
ZALG.DE 2023


## Training and Test Set

In [38]:
train_df = semantics_ftse100_df[semantics_ftse100_df['ESG Controversies Score Grade'] != 'Unknown']
print(train_df.shape[0])

16


In [39]:
test_df = semantics_ftse100_df[semantics_ftse100_df['ESG Controversies Score Grade'] == 'Unknown']
print(test_df.shape[0])

58


In [41]:
train_df

,firm,year,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,...,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,ESG Controversies Score,ESG Controversies Score Grade
0,1COV.DE,2022,0.130479,0.505199,0.600378,-0.477656,-0.174471,0.499294,-0.600625,-0.636520,0.701751,0.540555,0.268798,-0.253827,0.107586,-0.665974,-0.199498,0.608968,0.484472,0.546881,-0.455419,0.419032,-0.334159,-0.415122,0.410068,-0.541447,-0.625590,-0.478560,0.384087,0.253635,0.066590,0.259534,0.352463,-0.620511,-0.264266,0.691462,0.460079,0.475398,-0.475996,0.240907,0.694434,0.533751,-0.339226,-0.616026,-0.414357,-0.358934,0.053088,0.686750,0.528950,-0.664712,...,0.751445,-0.223564,-0.745369,0.458091,0.760691,-0.220254,-0.432880,0.011229,-0.588437,0.380800,0.491015,0.021233,-0.523250,-0.493968,0.534216,0.531353,-0.315117,-0.424845,-0.411290,0.729978,0.693388,-0.173789,0.244618,-0.594603,0.156404,0.510495,-0.580846,0.240752,-0.636583,0.470717,-0.393042,0.403760,0.683070,0.651256,-0.002697,-0.372086,0.667601,0.316503,-0.196795,0.014009,-0.405986,0.297155,-0.619607,0.714014,-0.243243,0.335700,0.734606,0.024849,100.000000,A+
2,ADSGn.DE,2022,-0.346749,-0.002502,-0.141569,0.135706,0.353608,0.097024,-0.032066,-0.205860,0.105043,-0.237411,-0.217469,0.426901,0.356096,-0.258442,0.258586,0.074039,0.088957,0.180499,-0.118204,0.249712,0.274465,0.371781,0.211265,-0.254681,-0.175135,0.370811,-0.237425,0.411039,0.383432,0.452075,-0.097054,-0.176692,-0.513129,0.364122,-0.334151,-0.255884,0.056133,-0.106999,0.378924,0.171222,-0.493156,-0.186640,0.293610,0.180465,0.374449,0.419876,-0.326793,-0.292623,...,0.335701,-0.417782,-0.096500,0.024978,0.432765,0.420547,-0.096433,0.245783,-0.209278,-0.040470,0.121749,-0.283628,-0.075871,-0.412947,-0.007121,0.021476,0.133151,0.059132,0.352613,0.219945,0.082368,-0.324195,-0.189520,-0.252642,-0.293246,-0.332109,-0.144255,-0.307813,-0.080864,-0.170644,0.306919,-0.167017,-0.094815,0.231349,0.235620,0.264082,0.342874,-0.279262,-0.300725,-0.193226,0.398977,0.402939,-0.592786,0.497523,-0.205579,-0.367670,0.285578,0.283730,2.631579,D-
4,AIR.PA,2022,-0.009459,0.158848,0.107230,-0.116253,-0.017543,0.143145,-0.414304,-0.227981,0.226929,0.266080,-0.049320,0.136348,0.015874,-0.267126,-0.188920,0.175482,0.083583,0.218538,-0.155100,0.356284,-0.080574,0.143410,0.160522,-0.227099,-0.281985,0.090259,-0.018785,0.135328,0.059879,0.227551,0.017598,-0.244785,-0.230742,0.324087,-0.048866,0.167740,-0.136329,0.239815,0.353576,0.173631,-0.197827,-0.254636,-0.095912,0.010695,0.115179,0.313809,0.090760,-0.275258,...,0.358406,-0.144852,-0.339112,0.133578,0.397854,0.171527,-0.199382,-0.033716,-0.189325,0.080698,0.166785,0.056231,-0.478562,-0.246668,0.103644,0.185576,-0.248937,-0.034875,0.137854,0.313294,0.255414,-0.121875,-0.026806,-0.255959,-0.089685,-0.049401,-0.195143,-0.044205,-0.250434,0.097936,0.028823,-0.003668,0.408344,0.296337,-0.076953,0.050862,0.292144,-0.048602,-0.108110,0.030223,0.149870,0.129567,-0.456120,0.409820,-0.085083,-0.094109,0.392943,-0.039840,100.000000,A+
8,BASFn.DE,2022,-0.093851,0.316798,0.394663,-0.179698,-0.005190,0.359180,-0.383086,-0.451957,0.503215,0.200838,0.108476,0.022878,0.197820,-0.541348,-0.030145,0.427929,0.366038,0.380638,-0.322680,0.364893,-0.072365,-0.048261,0.338824,-0.414706,-0.471719,-0.066275,0.164216,0.348455,0.196282,0.337106,0.136324,-0.451659,-0.353774,0.558783,0.165301,0.135812,-0.267792,0.074149,0.578637,0.417402,-0.402213,-0.452889,-0.093189,-0.329319,0.176247,0.545070,0.161818,-0.505579,...,0.598835,-0.304157,-0.468494,0.316234,0.641307,0.001291,-0.341353,0.105179,-0.469579,0.213895,0.360113,-0.081889,-0.292330,-0.489901,0.331082,0.346972,-0.157684,-0.242779,-0.034347,0.558329,0.516327,-0.313789,0.024677,-0.472344,0.023013,0.100771,-0.406906,0.008665,-0.505731,0.306090,-0.142579,0.260515,0.33

In [40]:
train_df.to_csv('../data/ML_Data/train_DAX_contains.csv', index=None)
test_df.to_csv('../data/ML_Data/test_DAX_contains.csv', index=None)